<a href="https://colab.research.google.com/github/IVIo3e/Fundamental-concepts-of-AI/blob/main/assignment02_habr_visual_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

#Загрузка и знакомство с данными
Для работы вам понадобятся предобработанные данные нашего учебного конкурса на kaggle «Прогноз популярности статьи на Хабре». Скачайте данные соревнования (данные были удалены с Kaggle ради организации последующего идентичного соревнования, так что тут ссылка на Google Drive).

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/data/howpop_train.csv")

In [4]:
df.shape

(134137, 17)

In [5]:
df.head(3).T

,0,1,2
url,https://habrahabr.ru/post/18284/,https://habrahabr.ru/post/18285/,https://habrahabr.ru/post/18286/
domain,habrahabr.ru,habrahabr.ru,habrahabr.ru
post_id,18284,18285,18286
published,2008-01-01 18:19:00,2008-01-01 18:30:00,2008-01-01 18:34:00
author,@Tapac,@DezmASter,@DezmASter
flow,develop,design,design
polling,False,False,False
content_len,4305,7344,8431
title,Новогодний подарок блоггерам — WordPress 2.3.2,"Сумасшедшие яйца, или сервис для отслеживания ...","Сумасшедшие яйца, или сервис для отслеживания ..."
comments,0,1,47



Избавимся сразу от переменных, названия которых заканчиваются на _lognorm (нужны для соревнования на Kaggle). Выберем их с помощью filter() и удалим drop-ом:

In [6]:
df.drop(
    filter(lambda c: c.endswith("_lognorm"), df.columns),
    axis=1,  # axis = 1: столбцы
    inplace=True,
)  # избавляет от необходимости сохранять датасет

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
post_id,134137.0,181307.054265,81766.350702,18284.0,115565.0,182666.0,257401.0,314088.0
content_len,134137.0,7516.929699,8724.771640,1.0,2179.0,4949.0,9858.0,352471.0
comments,134137.0,39.625994,62.398958,0.0,7.0,19.0,48.0,2266.0
favs,134137.0,71.999866,145.854135,0.0,8.0,27.0,78.0,8718.0
views,134137.0,16631.013084,31479.819691,3.0,2100.0,7600.0,18700.0,1730000.0
votes_plus,133566.0,35.536888,42.461073,0.0,11.0,22.0,45.0,1466.0
votes_minus,133566.0,8.050035,11.398282,0.0,2.0,5.0,10.0,445.0


In [8]:
df.describe(include=["object", "bool"]).T  # бинарные и категориальные переменные

,count,unique,top,freq
url,134137,134137,https://habrahabr.ru/post/162115/,1
domain,134137,2,habrahabr.ru,97048
published,134137,130291,2011-06-14 15:52:00,39
author,97657,22077,@alizar,5292
flow,97048,6,develop,53318
polling,134137,2,False,129412
title,134137,133628,"Интересные события, произошедшие в выходные",56


In [9]:
# настройка внешнего вида графиков в seaborn
sns.set_style("dark")
sns.set_palette("RdBu")
sns.set_context(
    "notebook", font_scale=1.5, rc={"figure.figsize": (15, 5), "axes.titlesize": 18}
)


Столбец published (время публикации) содержит строки. Чтобы мы могли работать с этими данными как с датой/временем публикации, приведём их к типу datetime:

In [10]:
print(df.published.dtype)
df["published"] = pd.to_datetime(df.published, yearfirst=True)
print(df.published.dtype)

object
datetime64[ns]



Создадим несколько столбцов на основе данных о времени публикации:

In [11]:
df["year"] = [d.year for d in df.published]
df["month"] = [d.month for d in df.published]

df["dayofweek"] = [d.isoweekday() for d in df.published]
df["hour"] = [d.hour for d in df.published]


#1. В каком месяце (и какого года) было больше всего публикаций?
* март 2016
* март 2015
* апрель 2015
* апрель 2016

In [13]:
a = [df[(df['month'] == 3) & (df['year'] == 2016)].shape[0],
     df[(df['month'] == 3) & (df['year'] == 2015)].shape[0],
     df[(df['month'] == 4) & (df['year'] == 2015)].shape[0],
     df[(df['month'] == 4) & (df['year'] == 2016)].shape[0]]

In [23]:
a.index(max(a))+1

2

Ответ -> март 2015

#2. Проанализируйте публикации в месяце из предыдущего вопроса
Выберите один или несколько вариантов:

* Один или несколько дней сильно выделяются из общей картины
* На хабре всегда больше статей, чем на гиктаймсе
* По субботам на гиктаймс и на хабрахабр публикуют примерно одинаковое число статей


Подсказки: постройте график зависимости числа публикаций от дня; используйте параметр hue; не заморачивайтесь сильно с ответами и не ищите скрытого смысла :)

In [28]:
df[(df['month'] == 3) & (df['year'] == 2015)].groupby('dayofweek').mean()

,post_id,polling,content_len,comments,favs,views,votes_plus,votes_minus,year,month,hour
dayofweek,,,,,,,,,,,
1,257969.285354,0.025253,8284.022727,18.078283,76.404040,16532.252525,20.186869,3.487374,2015.0,3.0,14.083333
2,257843.919517,0.060362,7773.134809,19.171026,74.410463,17596.535211,20.195171,3.531187,2015.0,3.0,13.424547
3,257453.132597,0.044199,7947.071823,19.939227,79.212707,17705.165746,21.743094,2.906077,2015.0,3.0,13.439227
4,257189.491379,0.045977,8131.589080,19.804598,90.718391,17739.114943,21.011494,3.344828,2015.0,3.0,13.298851
5,257781.523810,0.092063,8380.158730,20.425397,67.342857,16952.841270,18.504762,3.869841,2015.0,3.0,13.041270
6,255570.860870,0.034783,7008.591304,20.408696,82.686957,18831.304348,22.208696,4.547826,2015.0,3.0,14.617391
7,255507.820513,0.076923,11245.846154,24.897436,126.615385,21913.224359,26.794872,4.211538,2015.0,3.0,15.320513


#3. Когда лучше всего публиковать статью?
* Больше всего просмотров набирают статьи, опубликованные в 12 часов дня
* У опубликованных в 10 утра постов больше всего комментариев
* Больше всего просмотров набирают статьи, опубликованные в 6 часов утра
* Максимальное число комментариев на гиктаймсе набрала статья, опубликованная в 9 часов вечера
* На хабре дневные статьи комментируют чаще, чем вечерние

In [30]:
df.groupby('hour').mean()

,post_id,polling,content_len,comments,favs,views,votes_plus,votes_minus,year,month,dayofweek
hour,,,,,,,,,,,
0,168120.628831,0.036529,6824.571218,48.389066,77.037754,17546.982349,43.526601,9.677094,2012.199314,6.410395,3.837460
1,162485.714830,0.035567,6857.822801,49.417910,76.236583,17717.709114,43.329078,9.277051,2012.036520,6.397904,3.808828
2,163583.386444,0.042254,7036.796215,48.827025,82.805018,17328.690581,42.387040,9.294718,2012.078785,6.309859,3.805458
3,160226.364570,0.053273,7191.890886,53.123877,86.175225,18370.017972,45.887959,10.367675,2011.909499,6.530809,3.867137
4,157350.988182,0.045455,8080.478182,53.657273,95.343636,20678.389091,50.915146,9.882299,2011.903636,6.367273,3.732727
5,159812.991774,0.034078,8277.351351,47.816686,80.712103,17218.894242,42.232861,8.686761,2011.981199,6.613396,3.796710
6,176417.841603,0.044847,9173.211832,51.799618,90.884542,21354.310115,45.626195,8.858509,2012.504771,6.400763,3.383588
7,180619.888039,0.051460,8614.956885,43.263561,91.730181,19093.953408,40.033520,8.395251,2012.566064,6.598748,3.355355
8,190074.404762,0.048810,8509.013889,36.772222,73.224603,16014.419048,34.293556,7.521480,2012.866270,6.593651,3.248810


In [32]:
df[df['comments'] == max(df['comments'])]

,url,domain,post_id,published,author,flow,polling,content_len,title,comments,favs,views,votes_plus,votes_minus,year,month,dayofweek,hour
38763,https://geektimes.ru/post/123157/,geektimes.ru,123157,2011-07-01 02:35:00,@Dottore,NaN,False,2158,Как раздавать инвайты на Google+,2266,24,15300,226.0,55.0,2011,7,5,2


In [38]:
sum(df[(df['hour'] >= 6) | (df['hour'] < 18)]['comments']) / 12

442942.6666666667

In [39]:
sum(df[(df['hour'] < 6) | (df['hour'] >= 18)]['comments']) / 12

174821.08333333334

#4. Кого из топ-20 авторов чаще всего минусуют?
* @Mordatyj
* @Mithgol
* @alizar
* @ilya42

In [44]:
author = ["@Mordatyj", "@Mithgol", 
          "@alizar" ,"@ilya42"]

In [51]:
df[df["author"].isin(author)].groupby("author").mean()

,post_id,polling,content_len,comments,favs,views,votes_plus,votes_minus,year,month,dayofweek,hour
author,,,,,,,,,,,,
@Mithgol,129791.894643,0.037500,3105.428571,57.548214,41.121429,7809.033929,52.522523,20.481081,2010.994643,6.053571,3.573214,12.810714
@Mordatyj,65844.782051,0.000000,1199.426740,34.446886,4.677656,984.100733,26.068140,7.471455,2009.058608,6.457875,2.805861,13.934066
@alizar,171891.636810,0.012661,2906.113190,64.726190,40.983938,16812.196712,44.726444,7.928191,2012.348450,6.436886,3.486395,13.962963
@ilya42,182192.787109,0.003906,3214.958984,57.496094,97.871094,26201.613281,60.322266,6.216797,2012.771484,6.466797,3.656250,16.460938


#5. Сравните субботы и понедельники
Правда ли, что по субботам авторы пишут в основном днём, а по понедельникам — в основном вечером?

понедельник

In [67]:
df[(df["dayofweek"] == 1) & ((df['hour'] < 6) | (df['hour'] >= 18))].shape #вечер

(7505, 18)

In [68]:
df[(df["dayofweek"] == 1) & ((df['hour'] >= 6) | (df['hour'] < 18))].shape 

(23636, 18)

суббота

In [69]:
df[(df["dayofweek"] == 6) & ((df['hour'] < 6) | (df['hour'] >= 18))].shape #вечер

(4318, 18)

In [70]:
df[(df["dayofweek"] == 6) & ((df['hour'] >= 6) | (df['hour'] < 18))].shape

(8380, 18)